In [5]:
# Use if using colab
#from google.colab import drive
#drive.mount('/content/drive')

import argparse
import csv
import os
import sys
import re
import argparse
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from scipy.stats import loguniform
from scipy.io import loadmat
from sklearn import metrics
from torch.utils import data
from IPython.display import clear_output
# add current work directory in path
sys.path.append(os.getcwd())
from Data_Processing_Utils import Norm_each_sub_by_own_param,NormalizeData, train_val_test_split_df, PlotLoss, plot_confusion_matrix, \
    Get_Sub_Norm_params, windowing_Dataframe

from DATALOADERS import dataframe_dataset_triplet, Pandas_Dataset

from MODELS import MKCNN, MKCNN_grid, random_choice, train_model_triplet, pre_train_model_triplet, train_model_standard, MultiKernelConv2D_grid
pd.options.mode.chained_assignment = None

In [6]:
# custom pandas dataset class modified to provide also the angle
# used for multi-model testing
class Pandas_Dataset_angle(data.Dataset):

    def __init__(self, df_grouped_by_samples, return_sub=False):
        self.grouped = df_grouped_by_samples
        self.channels = [i for i in df_grouped_by_samples.obj.columns if 'ch' in i]
        self.indices = list(df_grouped_by_samples.indices)
        self.return_sub = return_sub

    def __len__(self):
        return len(self.grouped)

    def __getitem__(self, index):
        picked_smp = self.grouped.get_group(self.indices[index])
        # Picking only the channels columns from a single sample
        sample = torch.tensor(picked_smp.loc[:, self.channels].values).type(torch.float32)
        # picking only one label for each sample
        label = torch.tensor(picked_smp.loc[:, ['label']].head(1).values[0][0]).type(torch.int8)
        angle = torch.tensor(picked_smp.loc[:, ['angle']].head(1).values[0][0]).type(torch.int16)
        if self.return_sub:
            # picking only one subject
            sub = torch.tensor(picked_smp.loc[:, ['sub']].head(1).values[0][0]).type(torch.int8)
            return sample, label, angle, sub
        else:

            # It's missing the part in which I Normalize the data for each subject, or for each subject and channels
            # It's not a good idea to do that on the dataloader while producing results
            return sample, label, angle


In [7]:
#Function used to merge the dataframes
def make_merged_df(dfs_path: str, wnd_list: list, save = True, save_path: str = None):
    """
    dfs_path: folder where the windowed dataframes are stored
    wnd_list: list with the name of the windowed dfs to merge
    """
    print("Merging Angles Dataframes...")
    #wnd_list = [x for x in sorted(os.listdir(dfs_path)) if name in x]
    dfs = []
    prev_df_lenghts = 0
    i = 0
    # loop to merge all the windowed dataframes into one, while changing the sample_index accordingly
    for dataframe in wnd_list:
        df = pd.read_csv(dfs_path+dataframe)
        df['sample_index'] = df['sample_index'] + prev_df_lenghts +1*(i>0)
        df
        dfs.append(df)
        prev_df_lenghts = df['sample_index'].iloc[-1]
        i+=1

    total_df = pd.concat(dfs, ignore_index = True)
    #print(total_df.shape)
    total_df_chs = [col for col in total_df.columns if 'Ch' in col]
    total_df[total_df_chs] = total_df[total_df_chs].astype(np.uint16)
    total_df['angle'] = total_df['angle'].astype(np.int16)

    minAngle, maxAngle = total_df['angle'].abs().min(), total_df['angle'].abs().max()
    print(f"minAngle: {minAngle} maxAngle: {maxAngle}")
    angleRange = maxAngle - minAngle
    interval = 5*round(angleRange * 0.125/5) #round interval to the closest multiple of 5
    residue = angleRange-interval*8
    intervals = [abs(minAngle+interval), abs(minAngle+interval*3), abs(maxAngle-interval*3), abs(maxAngle-interval)]
    print(f"Angle Intervals: {intervals}")

    angle_index = []
    for ang in total_df['angle']:
        ang = abs(ang)
        if ang < intervals[0]:
            angle_index.append(1)
        elif ang < intervals[1]:
            angle_index.append(2)
        elif ang < intervals[2]:
            angle_index.append(3)
        elif ang < intervals[3]:
            angle_index.append(4)
        else:
            angle_index.append(5)

    if 'Unnamed: 0' in total_df.columns:
        total_df.drop('Unnamed: 0', axis = 1, inplace =True)
    total_df["angle_index"] = angle_index
    total_df["angle_index"] = total_df["angle_index"].astype(np.int8)

    if save:
        if not os.path.exists(save_path):
          os.makedirs(save_path)
        save_directory = (dfs_path if save_path == None else save_path) + 'dataframe_wnd_200_angle_merged.csv'
        total_df.to_csv(save_directory, index = False)
        print("\nMerged angles saved at ----> "+ save_directory+"\n")
    return total_df


In [26]:
#Data path
data_path = os.getcwd() + '/DATA/'

subjects = [ 'Sub2']
days = [1] #be sure that all the subject have the days you want to use



In [24]:
# Directory and data creation
# set window length and overlap for the windowing
WND_LEN = 200
overlap = 0.5
n_channels = 8
columns = [f'Ch{ch}' for ch in range(1, n_channels+1)]

# True if you want to generate, merge and modify the dataframes
# You have to do this the first time
operate_on_df = True 
# True if you want to generate the dataframes
make_dataframes = True
# True if you want to merge the days 1  and 2
merge_days = False
# True if you want to merge the angles
# It's advised to merge all the angles in one dataframe to save space and complexity
# A new column 'angle_index' will be added to keep track of which angle is which
merge_angles = True

In [27]:
# Dataframe creation and manipulation
for subj in subjects:
    for day in days:
        current_path = data_path+f'{subj}/day{day}/mat/'
        save_path = data_path+f'/{subj}/day{day}/Dataframe/'
        if operate_on_df:
            if make_dataframes:
                EMG_data = os.listdir(current_path)
                for name in EMG_data:
                    angle = re.search(r'%s(\d+)' % 'angle_', name).group(1)
                    mat = loadmat(current_path + name)
                    df = pd.DataFrame(data=mat['emg'], columns=columns, dtype=np.uint32)
                    df['Exe_Num'] = mat['relabel_stimulus'].astype(np.uint8)
                    df['Rep_Num'] = mat['relabel_repetition'].astype(np.uint8)
                    df['Sub'] = np.full(shape=len(df), fill_value=1).astype(np.uint8)
                    df['Angle'] = mat['angle'].astype(np.int16)
                    df = df.loc[df['Exe_Num'] != 0]     # Deleting rest position
                    df = df.loc[df['Rep_Num'] != 0]     # Deleting other, if remaining (shouldn't but some in day1 angle 4) resting position (?)
                    df['Exe_Num'] = df['Exe_Num'] - 1    # Starting label from 0
                    #df.to_csv(database + f'dataframe_ang_{angle}_no_rest.csv', columns=df.columns)
                    windowing_Dataframe(df, WND_LEN, overlap_perc=overlap, NinaPro_Database_Num=None, drop_last=False, keep_angle = True,
                                        path_to_save=save_path, filename=f'dataframe_wnd_{WND_LEN}_angle_{angle}.csv')
        
            # % Merging day1 and day2, saving day-type in 'sub' column because, being all the signals from same subject (Giulio Sartorato),
            # there is no need to keep sub column. Moreover, having some functions working for domain adaptation between subject,
            # using column subject to perform INTER-DAY analysis will become easier.
            if merge_days:
                print(f'Merging: -> dataframe_wnd_{WND_LEN}_angle_merged  of day1 & day2')
                path = data_path + f'{subj}/day1/Dataframe/'
                os.chdir(path)
                df1 = pd.read_csv(path + f'dataframe_wnd_{WND_LEN}_angle_merged.csv')
                path = data_paht + f'{subj}/day2/Dataframe/'
                os.chdir(path)
                df2 = pd.read_csv(path + f'dataframe_wnd_{WND_LEN}_angle_merged.csv')
        
                df2['sub'] = df2['sub'] + 1   # It will represent day2
                df2['sample_index'] = df2['sample_index'] + max(df1.sample_index) + 1   # Correcting sample_index to be unique for each window
        
                df = pd.concat([df1, df2])
                merge_save_dir = data_path + f'{subj}/merged_day1_2/'
                if not os.path.exists(merge_save_dir):
                    os.makedirs(merge_save_dir)
                df.to_csv(merge_save_dir + f'Dataframe/df_merged_wnd_{WND_LEN}_ang_merged.csv', columns=df.columns, index=False)
        
            if merge_angles:
                s_string = "_angle"
                df_list = [x for x in os.listdir(save_path) if s_string in x]
                print(df_list)
                make_merged_df(dfs_path=save_path,wnd_list=  df_list, save = True, save_path=save_path)
                for df in df_list:
                    os.remove(save_path+df)


Windowing subject: 1
Saved in -> C:\Users\sarto\OneDrive\Desktop\Git Repo\Model Training Code/DATA//Sub2/day1/Dataframe/  as -> dataframe_wnd_200_angle_1.csv
Windowing subject: 1
Saved in -> C:\Users\sarto\OneDrive\Desktop\Git Repo\Model Training Code/DATA//Sub2/day1/Dataframe/  as -> dataframe_wnd_200_angle_2.csv
Windowing subject: 1
Saved in -> C:\Users\sarto\OneDrive\Desktop\Git Repo\Model Training Code/DATA//Sub2/day1/Dataframe/  as -> dataframe_wnd_200_angle_3.csv
Windowing subject: 1
Saved in -> C:\Users\sarto\OneDrive\Desktop\Git Repo\Model Training Code/DATA//Sub2/day1/Dataframe/  as -> dataframe_wnd_200_angle_4.csv
Windowing subject: 1
Saved in -> C:\Users\sarto\OneDrive\Desktop\Git Repo\Model Training Code/DATA//Sub2/day1/Dataframe/  as -> dataframe_wnd_200_angle_5.csv
['dataframe_wnd_200_angle_1.csv', 'dataframe_wnd_200_angle_2.csv', 'dataframe_wnd_200_angle_3.csv', 'dataframe_wnd_200_angle_4.csv', 'dataframe_wnd_200_angle_5.csv']
Merging Angles Dataframes...
minAngle: 0 max